### Raspberry Pi

In [ ]:
import cv2
import socket
import struct
import pickle
import threading
import time

def webcam_stream(cam_index, server_ip, server_port, stop_event, cam_width, cam_height):
    try:
        # 소켓 생성 및 연결
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect((server_ip, server_port))
        connection = client_socket.makefile('wb')

        # 웹캠 연결
        cap = cv2.VideoCapture(cam_index)
        cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.75) 

        if not cap.isOpened():
            raise Exception(f"웹캠 {cam_index}을(를) 열 수 없습니다.")

        # 해상도 설정 (320x240)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, cam_width)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cam_height)

        prev_time = time.time()
        # fps = 0
        # frame_count = 0

        while not stop_event.is_set():  # 종료 이벤트가 설정되지 않았을 때만 루프 실행
            ret, frame = cap.read()
            if not ret:
                raise Exception(f"웹캠 {cam_index}에서 프레임을 읽을 수 없습니다.")

            # 프레임 카운트 증가
            # frame_count += 1

            # 현재 시간과 이전 시간을 비교해 FPS 계산
            # current_time = time.time()
            # elapsed_time = current_time - prev_time
            # if elapsed_time >= 1.0:  # 1초마다 FPS 계산
            #     fps = frame_count / elapsed_time
            #     frame_count = 0
            #     prev_time = current_time

            # 프레임에 FPS 표시
            # cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # 프레임 직렬화
            data = pickle.dumps(frame)
            size = len(data)

            # 프레임을 서버로 전송 (길이 정보를 함께 전송)
            client_socket.sendall(struct.pack(">L", size) + data)
            
            # 1초당 30프레임 제한
            time.sleep(1/30)

    except Exception as e:
        print(f"웹캠 {cam_index}에서 오류 발생: {e}")
    finally:
        cap.release()
        connection.close()
        client_socket.close()

def check_keypress(stop_event):
    while not stop_event.is_set():
        if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' 키가 눌리면 종료
            stop_event.set()

if __name__ == '__main__':
    # Server 1과 Server 2의 IP와 포트 설정
    server1_ip = "192.168.0.140"  # Server 1의 IP
    server1_port = 9002           # Server 1의 포트

    server2_ip = "192.168.0.39"  # Server 2의 IP
    server2_port = 9003           # Server 2의 포트

    # 종료 이벤트 설정
    stop_event = threading.Event()

    # 두 개의 웹캠을 각각의 서버로 스트리밍하는 스레드 생성
    webcam1_thread = threading.Thread(target=webcam_stream, args=(0, server1_ip, server1_port, stop_event, 640, 480 ))
    webcam2_thread = threading.Thread(target=webcam_stream, args=(1, server2_ip, server2_port, stop_event, 640, 480 ))

    # 키 입력 감지용 스레드 시작
    keypress_thread = threading.Thread(target=check_keypress, args=(stop_event,))
    keypress_thread.start()

    # 스레드 시작
    webcam1_thread.start()
    webcam2_thread.start()

    # 스레드가 종료될 때까지 대기
    webcam1_thread.join()
    webcam2_thread.join()

    # 키 입력 감지 스레드 종료
    keypress_thread.join()

### Pollination_server and client

In [ ]:
import cv2
import socket
import struct
import pickle

def receive_stream(server_ip, server_port):
    # 소켓 생성 및 바인딩
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((server_ip, server_port))
    server_socket.listen(1)
    print(f"서버가 {server_ip}:{server_port}에서 대기 중입니다...")

    # 클라이언트 연결 수립
    conn, addr = server_socket.accept()
    print(f"클라이언트 {addr}와 연결되었습니다.")

    # 파일 객체 생성
    file = conn.makefile('rb')

    try:
        while True:
            # 프레임 크기 수신
            packed_size = file.read(struct.calcsize(">L"))
            if not packed_size:
                break
            size = struct.unpack(">L", packed_size)[0]

            # 프레임 데이터 수신
            data = b""
            while len(data) < size:
                data += file.read(size - len(data))

            # 프레임 디코딩
            frame = pickle.loads(data)
            if frame is None:
                break

            # 프레임을 화면에 표시
            cv2.imshow("Received Video", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except Exception as e:
        print(f"수신 중 오류 발생: {e}")
    finally:
        file.close()
        conn.close()
        server_socket.close()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    # 서버의 IP와 포트 설정
    server_ip = "0.0.0.0"  # 모든 네트워크 인터페이스에서 수신
    server_port = 9003     # 서버 포트 (클라이언트에서 지정한 포트와 동일해야 함)

    # 영상 수신 함수 호출
    receive_stream(server_ip, server_port)

In [ ]:
import cv2
import socket
import struct
import pickle
from ultralytics import YOLO  # YOLOv8 모델을 사용하기 위한 패키지

def process_frame(model, frame):
    # YOLOv8 모델을 사용하여 객체 감지 수행
    results = model(frame)
    # 결과를 프레임에 표시
    annotated_frame = results.render()[0]  # 결과를 렌더링
    return annotated_frame

def receive_stream(server_ip, server_port):
    # 소켓 생성 및 바인딩
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((server_ip, server_port))
    server_socket.listen(1)
    print(f"서버가 {server_ip}:{server_port}에서 대기 중입니다...")

    # 클라이언트 연결 수립
    conn, addr = server_socket.accept()
    print(f"클라이언트 {addr}와 연결되었습니다.")

    # 파일 객체 생성
    file = conn.makefile('rb')

    # YOLO 모델 로드
    model = YOLO('/home/jh/project/deeplearning_project/add_pollination/yolo/runs/detect/total_apple_flower_yolov8m_batch8_epochs200/weights/best.pt')

    try:
        while True:
            # 프레임 크기 수신
            packed_size = file.read(struct.calcsize(">L"))
            if not packed_size:
                break
            size = struct.unpack(">L", packed_size)[0]

            # 프레임 데이터 수신
            data = b""
            while len(data) < size:
                data += file.read(size - len(data))

            # 프레임 디코딩
            frame = pickle.loads(data)
            if frame is None:
                break

            # YOLOv8 모델을 사용하여 프레임 처리
            annotated_frame = process_frame(model, frame)

            # 프레임을 화면에 표시
            cv2.imshow("YOLOv8 Object Detection", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except Exception as e:
        print(f"수신 중 오류 발생: {e}")
    finally:
        file.close()
        conn.close()
        server_socket.close()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    # 서버의 IP와 포트 설정
    server_ip = "0.0.0.0"  # 모든 네트워크 인터페이스에서 수신
    server_port = 9002     # 서버 포트 (클라이언트에서 지정한 포트와 동일해야 함)

    # 영상 수신 함수 호출
    receive_stream(server_ip, server_port)


In [ ]:
import cv2
import socket
import struct
import pickle
from ultralytics import YOLO  # YOLOv8 모델을 사용하기 위한 패키지

def load_model():
    # YOLOv8 모델 로드
    model = YOLO('yolov8.pt')  # 'yolov8.pt'는 YOLOv8 모델 파일 경로
    return model

def process_frame(model, frame):
    # YOLOv8 모델을 사용하여 객체 감지 수행
    results = model(frame)
    # 결과를 프레임에 표시
    annotated_frame = results.render()[0]  # 결과를 렌더링

    # 감지된 객체가 있는지 확인
    if len(results.pandas().xyxy[0]) > 0:  # 감지된 객체가 있을 때
        cv2.putText(annotated_frame, 'Object Detected', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    return annotated_frame

def receive_stream(server_ip, server_port):
    # 소켓 생성 및 바인딩
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((server_ip, server_port))
    server_socket.listen(1)
    print(f"서버가 {server_ip}:{server_port}에서 대기 중입니다...")

    # 클라이언트 연결 수립
    conn, addr = server_socket.accept()
    print(f"클라이언트 {addr}와 연결되었습니다.")

    # 파일 객체 생성
    file = conn.makefile('rb')

    # YOLO 모델 로드
    model = load_model()

    try:
        while True:
            # 프레임 크기 수신
            packed_size = file.read(struct.calcsize(">L"))
            if not packed_size:
                break
            size = struct.unpack(">L", packed_size)[0]

            # 프레임 데이터 수신
            data = b""
            while len(data) < size:
                data += file.read(size - len(data))

            # 프레임 디코딩
            frame = pickle.loads(data)
            if frame is None:
                break

            # YOLOv8 모델을 사용하여 프레임 처리
            annotated_frame = process_frame(model, frame)

            # 프레임을 화면에 표시
            cv2.imshow("YOLOv8 Object Detection", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except Exception as e:
        print(f"수신 중 오류 발생: {e}")
    finally:
        file.close()
        conn.close()
        server_socket.close()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    # 서버의 IP와 포트 설정
    server_ip = "0.0.0.0"  # 모든 네트워크 인터페이스에서 수신
    server_port = 9002     # 서버 포트 (클라이언트에서 지정한 포트와 동일해야 함)

    

    # 영상 수신 함수 호출
    receive_stream(server_ip, server_port)


In [1]:
import cv2
import socket
import struct
import pickle
import json
from ultralytics import YOLO

# 서버의 IP와 포트 설정
server_ip = "172.30.1.31"  # 모든 네트워크 인터페이스에서 수신
server_port = 9003     # 서버 포트 (클라이언트에서 지정한 포트와 동일해야 함)

# 감지된 객체 정보를 전송할 서버의 IP와 포트 설정
detection_server_ip = "172.30.1.62"  # 감지 정보 전송 서버의 IP
detection_server_port = 8888       # 감지 정보 전송 서버의 포트

# 소켓 생성 및 바인딩
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((server_ip, server_port))
server_socket.listen(1)
print(f"서버가 {server_ip}:{server_port}에서 대기 중입니다...")

# 클라이언트 연결 수립
conn, addr = server_socket.accept()
print(f"클라이언트 {addr}와 연결되었습니다.")

# 파일 객체 생성
file = conn.makefile('rb')

# YOLO 모델 로드
model = YOLO('/home/jh/project/deeplearning_project/add_pollination/yolo/runs/detect/total_apple_flower_yolov8m_batch8_epochs200/weights/best.pt')

try:
    while True:
        # 프레임 크기 수신
        packed_size = file.read(struct.calcsize(">L"))
        if not packed_size:
            break
        size = struct.unpack(">L", packed_size)[0]

        # 프레임 데이터 수신
        data = b""
        while len(data) < size:
            data += file.read(size - len(data))

        # 프레임 디코딩
        frame = pickle.loads(data)
        if frame is None:
            break

        # YOLOv8 모델을 사용하여 객체 감지 수행
        results = model(frame)
        annotated_frame = results.render()[0]

        # 감지된 객체 정보 추출
        detections = []
        for _, row in results.pandas().xyxy[0].iterrows():
            detection = {
                'class' : int(row['class']),
                'class_name' : results.names[int(row['class'])],
                'confidence' : row['confidence'],
                'xmin' : row['xmin'],
                'ymin' : row['ymin'],
                'xmax' : row['xmax'],
                'ymax' : row['ymax']
            }
            detection.append(detection)

        
        # 프레임을 화면에 표시
        cv2.imshow("pollination detect", annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if detections:
            try:
                # central_server 연결
                client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                client_socket.connect((detection_server_ip, detection_server_port))

                # 감지된 객체 정보를 Json 형식으로 직렬화
                data = json.dump(detections)
                # 패킷 시작에 "TS" 추가, 끝에 '\n' 추가
                packet = f"""TS{data}\n"""
                packet = packet.encode('utf-8')

                # 데이터 전송 
                client_socket.sendall(packet)

            except Exception as e:
                print(f"감지 정보 전송 중 오류 발생: {e}")
            finally:
                client_socket.close()


except Exception as e:
    print(f"수신 중 오류 발생: {e}")
finally:
    file.close()
    conn.close()
    server_socket.close()

OSError: [Errno 98] Address already in use

In [ ]:
import cv2
import socket
import struct
import pickle
import json
from ultralytics import YOLO

def load_model():
    # YOLOv8 모델 로드
    model = YOLO('yolov8.pt')  # 'yolov8.pt'는 YOLOv8 모델 파일 경로
    return model

def process_frame(model, frame):
    # YOLOv8 모델을 사용하여 객체 감지 수행
    results = model(frame)
    
    # 감지된 객체 정보 추출
    detections = []
    for _, row in results.pandas().xyxy[0].iterrows():
        detection = {
            'class': int(row['class']),
            'class_name': results.names[int(row['class'])],
            'confidence': row['confidence'],
            'xmin': row['xmin'],
            'ymin': row['ymin'],
            'xmax': row['xmax'],
            'ymax': row['ymax']
        }
        detections.append(detection)
    
    return frame, detections

def send_detections(detections, server_ip, server_port):
    try:
        # 소켓 생성 및 연결
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect((server_ip, server_port))
        
        # 감지된 객체 정보를 JSON 형식으로 직렬화
        data = json.dumps(detections)
        # 패킷 시작에 'TS' 추가, 끝에 '\n' 추가
        packet = f"TS{data}\n"
        packet = packet.encode('utf-8')
        size = len(packet)
        
        # 데이터 전송 (길이 정보와 함께 전송)
        client_socket.sendall(struct.pack(">L", size) + packet)
    except Exception as e:
        print(f"감지 정보 전송 중 오류 발생: {e}")
    finally:
        client_socket.close()

def receive_stream(server_ip, server_port, detection_server_ip, detection_server_port):
    # 소켓 생성 및 바인딩
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((server_ip, server_port))
    server_socket.listen(1)
    print(f"서버가 {server_ip}:{server_port}에서 대기 중입니다...")

    # 클라이언트 연결 수립
    conn, addr = server_socket.accept()
    print(f"클라이언트 {addr}와 연결되었습니다.")

    # 파일 객체 생성
    file = conn.makefile('rb')

    # YOLO 모델 로드
    model = load_model()

    try:
        while True:
            # 프레임 크기 수신
            packed_size = file.read(struct.calcsize(">L"))
            if not packed_size:
                break
            size = struct.unpack(">L", packed_size)[0]

            # 프레임 데이터 수신
            data = b""
            while len(data) < size:
                data += file.read(size - len(data))

            # 프레임 디코딩
            frame = pickle.loads(data)
            if frame is None:
                break

            # YOLOv8 모델을 사용하여 프레임 처리
            processed_frame, detections = process_frame(model, frame)

            # 감지된 객체 정보를 다른 서버로 전송
            if detections:  # 감지된 객체가 있을 때만 전송
                send_detections(detections, detection_server_ip, detection_server_port)

    except Exception as e:
        print(f"수신 중 오류 발생: {e}")
    finally:
        file.close()
        conn.close()
        server_socket.close()

if __name__ == '__main__':
    # 서버의 IP와 포트 설정
    server_ip = "0.0.0.0"  # 모든 네트워크 인터페이스에서 수신
    server_port = 9002     # 서버 포트 (클라이언트에서 지정한 포트와 동일해야 함)

    # 감지된 객체 정보를 전송할 서버의 IP와 포트 설정
    detection_server_ip = "192.168.0.100"  # 감지 정보 전송 서버의 IP
    detection_server_port = 9004           # 감지 정보 전송 서버의 포트

    # 영상 수신 함수 호출
    receive_stream(server_ip, server_port, detection_server_ip, detection_server_port)
